In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window as W

In [2]:
from pyspark.sql.types import DateType,StringType

In [3]:
spark = SparkSession.builder.appName("Test").getOrCreate()

In [4]:
student_data = [("Vendor1",10,"cat1",4,"sub_cat1",1),
                ("Vendor1",10,"cat1",4,"sub_cat2",2),
                ("Vendor1",10,"cat1",4,"sub_cat3",3),
                ("Vendor1",10,"cat1",4,"sub_cat4",4)]

columns = ["vendor_name","count","catogory","category_count","sub_cat","sub_cat_count"]

In [5]:
dataframe = spark.createDataFrame(data = student_data,schema = columns)
dataframe.show()

+-----------+-----+--------+--------------+--------+-------------+
|vendor_name|count|catogory|category_count| sub_cat|sub_cat_count|
+-----------+-----+--------+--------------+--------+-------------+
|    Vendor1|   10|    cat1|             4|sub_cat1|            1|
|    Vendor1|   10|    cat1|             4|sub_cat2|            2|
|    Vendor1|   10|    cat1|             4|sub_cat3|            3|
|    Vendor1|   10|    cat1|             4|sub_cat4|            4|
+-----------+-----+--------+--------------+--------+-------------+



In [102]:
conf = {
    'header' : True,
    'inferSchema' : True,
    
    
}

In [104]:
file_path = "E:\workforce\spark\data\Bengaluru_House_Data*.csv"
df = spark.read.format('csv').options(**conf).option('path',file_path).load()
df.show()

+--------------------+-------------+--------------------+---------+-------+----------+----+-------+-----+
|           area_type| availability|            location|     size|society|total_sqft|bath|balcony|price|
+--------------------+-------------+--------------------+---------+-------+----------+----+-------+-----+
|Super built-up  Area|       19-Dec|Electronic City P...|    2 BHK|Coomee |      1056|   2|      1|39.07|
|          Plot  Area|Ready To Move|    Chikka Tirupathi|4 Bedroom|Theanmp|      2600|   5|      3|120.0|
|      Built-up  Area|Ready To Move|         Uttarahalli|    3 BHK|   NULL|      1440|   2|      3| 62.0|
|Super built-up  Area|Ready To Move|  Lingadheeranahalli|    3 BHK|Soiewre|      1521|   3|      1| 95.0|
|Super built-up  Area|Ready To Move|            Kothanur|    2 BHK|   NULL|      1200|   2|      1| 51.0|
|Super built-up  Area|Ready To Move|          Whitefield|    2 BHK|DuenaTa|      1170|   2|      1| 38.0|
|Super built-up  Area|       18-May|    Old Ai

In [106]:
df.rdd.getNumPartitions()

1

In [109]:
df.groupBy("area_type").count().show()

+--------------------+-----+
|           area_type|count|
+--------------------+-----+
|      Built-up  Area| 2418|
|Super built-up  Area| 8790|
|          Plot  Area| 2025|
|        Carpet  Area|   87|
+--------------------+-----+



In [110]:
save_path = filepath = "E:/workforce/spark/data/hudsonu-product/tmp/"
df.write.format("csv").mode('overwrite').option("path",save_path).save()

In [112]:
new_df = df.repartition(4,'area_type')

In [113]:
new_df.rdd.getNumPartitions()

4

In [115]:
new_df.write.format('csv').mode('overwrite').option("path",save_path).\
partitionBy("area_type").option("maxRecordsPerFile",1000).save()

In [116]:
df44 = spark.read.format('csv').options(**conf).option('path',save_path).load().filter(F.col('area_type') == 'Carpet  Area')

In [118]:
new_df.write.format("csv").mode('overwrite').option("path",save_path).\
bucketBy(3,"bath").saveAsTable("bucketTable")